<a href="https://colab.research.google.com/github/Sundaynot/air-quality/blob/main/src/db_initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
print ("hello")

hello
